# 猫狗大战 毕业项目——Fine-tuning ResNet50


## 开始
导入一切并我们设置所使用的GPU。
- dev0: GTX1070Ti

In [1]:
## Denpendencies
import os
import csv
import cv2
import h5py
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm  
from time import time
from PIL import Image
from functions_collection import *

## Keras
from keras.applications.xception import decode_predictions
from keras.applications.xception import preprocess_input
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing import image
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

## GPU choice
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Specific setting for CUDNN_STATUS_INTERNAL_ERROR
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

Using TensorFlow backend.


## 基准模型
作为迁移学习的基础，这里我们使用ResNet50为基准模型：
- [ResNet](https://arxiv.org/abs/1512.03385)

在导出预训练模型特征时，我们所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

我们首先冻结所有Resnet的权重参数，只训练全链接层。我们在融合模型中已经导出了所有训练数据和测试数据在Resnet上的特征，基于这些特征，我们训练猫狗问题的网络。

In [2]:
train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('trainAlter')
os.mkdir('trainAlter/cat')
os.mkdir('trainAlter/dog')

rmrf_mkdir('testAlter')
os.symlink('../test/', 'testAlter/test')

for filename in train_cat:
    os.symlink('../../train/'+filename, 'trainAlter/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'trainAlter/dog/'+filename)

In [3]:
#为数据连理symbol-link
train_data_dir, valid_data_dir, test_data_dir = symbol_link()

100%|██████████| 12500/12500 [00:00<00:00, 68409.36it/s]


In [4]:
#导入训练数据和测试数据
X_train, Y_train, X_test = export_data("data_resnet50.h5")

In [5]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
#训练模型并导出权重参数
filepath="resnet50.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 53us/step - loss: 0.2975 - acc: 0.8705 - val_loss: 0.1111 - val_acc: 0.9648
Epoch 2/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.1279 - acc: 0.9514 - val_loss: 0.0793 - val_acc: 0.9718
Epoch 3/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.1001 - acc: 0.9607 - val_loss: 0.0691 - val_acc: 0.9738
Epoch 4/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.0853 - acc: 0.9670 - val_loss: 0.0641 - val_acc: 0.9772
Epoch 5/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.0838 - acc: 0.9692 - val_loss: 0.0609 - val_acc: 0.9780
Epoch 6/20
20000/20000 [==============================] - 0s 17us/step - loss: 0.0772 - acc: 0.9705 - val_loss: 0.0591 - val_acc: 0.9782
Epoch 7/20

In [7]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-resnet50.csv")

12500/12500 [==============================] - 0s 18us/step
Found 12500 images belonging to 1 classes.


这一模型在Kaggle上的得分为0.07529，还不错。下一步我们将开始Fine-tuning基于Resnet50的猫狗分类模型。